In [ ]:
import numpy as np
import pandas as pd
import math
import scipy
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import PLtools
import time
import matplotlib.cm as cm
import matplotlib as mpl
import os
from os import listdir
%matplotlib inline

#Constants
pi = math.pi
heV = 4.14e-15 #eV*s
c = 2.99792e8 #m/s
kbeV = 8.6173e-5 #eV/K
keV = 8.6173e-5 #eV/K
h = 6.626e-34
kb = 1.38065e-23
q = 1.60218e-19

default_figsize = mpl.rcParamsDefault['figure.figsize']
mpl.rcParams['figure.figsize'] = [1.5*val for val in default_figsize]
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 24}

mpl.rc('font', **font)
mpl.rc('axes', linewidth=3)

In [34]:
#SPECIFY ANALYSIS DIRECTORY HERE
directory = '2018_7-24EL'

names_InGaAs = []
names_Si = []


for file in listdir(directory):
    if (file[-4:] == '.txt') and not ('Dark' in file) and not ('Sidetector' in file):
        names_InGaAs.append(file)
    elif (file[-4:] == '.txt') and not ('Dark' in file) and ('Sidetector' in file):
        names_Si.append(file)

names_Si

['2018_7-24_Sample10_MW_1p1V_Sidetector.txt',
 '2018_7-24_Sample12_MW_1p1V_Sidetector.txt',
 '2018_7-24_Sample13_MW_1p1V_Sidetector.txt',
 '2018_7-24_Sample14_MW_1p1V_Sidetector.txt',
 '2018_7-24_Sample14_MW_1p5V_Sidetector.txt',
 '2018_7-24_Sample14_MW_1p5V_Sidetector2.txt',
 '2018_7-24_Sample15_MW_1p1V_Sidetector.txt',
 '2018_7-24_Sample16_MW_1p1V_Sidetector.txt',
 '2018_7-24_Sample17_MW_1p1V_Sidetector.txt',
 '2018_7-24_Sample9_MW_1p1V_Sidetector.txt']

In [11]:
def AIPL_maps(lname,dname,grating,laser_power,ND_filter,elpl):
    """
    This function analyzes AIPL gradient XY maps and outputs numpy arrays of 
    meaningful parameters
    
    INPUTS:
    lname = name corresponding to datafile (expecting an XY map, .txt)
    dname = name corresponding to dark file (expecting single spectrum, .txt)
    laswer_power = laswer power in SI [W]
    ND_filter = OD filter placed between emission and detection (usually 0.0)
    
    OUTPUTS:
    meanPL, PLQY, chi, QFLS, etc. shaped in same way as gradient. Expecting 10x50 gradient aspect
    """
    if dname != []:
        df = pd.read_table(dname,header=None)
        dark = df.values
    else:
        dark = []
    df = pd.read_table(lname,header=None)
    data = df.values



    AIPL = PLtools.aipl(data,dark,grating)
    #Fix this issue - shouldn't have to load twice
    df = pd.read_table(lname,header=None)
    raw_data = df.values
    #print(raw_data)

#    for ii in range(500):
#        #plt.style.use('seaborn')
#        plt.plot(AIPL[0,:],AIPL[ii+1,:])
#        plt.xlabel('nm', fontsize=18)
#        plt.ylabel('AIPL',fontsize=18)

    
    aipl_data = AIPL

    DiodeReadings_1sun = laser_power
    DiodeResponse532= 0.2741
    Area785ImageJ = pi*(6.01e-6)**2
    Ep532 = 2.3305 #E per photon @532

    #Load data from Mathmatica calcs to determine SQ limits @ 300 K and 350 K for various
    #Egs
    Egs = np.loadtxt('../../data/PLdata/vocmax_data/Egs.txt')
    VocSQs300 = np.loadtxt('../../data/PLdata/vocmax_data/VocMaxs.txt') # 300 K
    Jphs = np.loadtxt('../../data/PLdata/vocmax_data/Jphs.txt') #300 K
    VocSQs350 = np.loadtxt('../../data/PLdata/vocmax_data/VocMaxs2.txt') # 350 K
    VocSQs300_fn = interp1d(Egs, VocSQs300, kind='cubic')
    VocSQs350_fn = interp1d(Egs, VocSQs350, kind='cubic')
    Jphs_fn = interp1d(Egs, Jphs, kind='cubic')


    DiodeReading = DiodeReadings_1sun
    P532 = DiodeReading/(DiodeResponse532*Area785ImageJ*10) #W/m^2
    Jp532 = DiodeReading*0.925/(DiodeResponse532*Area785ImageJ*1.60218e-19*Ep532*2)

    if data.shape[1] == 2: #single spectrum
        lam = aipl_data[:,0]
        E = heV*c/(lam*1e-9)
        if elpl == 0:
            Ipl = aipl_data[:,1] 
        else:
            Ipl = aipl_data[:,1]*(6.01e-6*2/2.2e-3)**2
        """
        maxI = np.max(Ipl)
        maxI_idx = np.argmax(Ipl)
        peak_pos = E[maxI_idx]
        HHMax_idx = np.argmin(np.absolute(maxI/2-Ipl[:maxI_idx]))
        LHMax_idx = np.argmin(np.absolute(maxI/2-Ipl[maxI_idx:]))
        LHMax_idx = LHMax_idx+maxI_idx-1
        FWHM = E[HHMax_idx]-E[LHMax_idx]
        VocSQ300 = VocSQs300_fn(E[maxI_idx])
        VocSQ350 = VocSQs350_fn(E[maxI_idx])    
        JphSQ = Jphs_fn(E[maxI_idx])
        NSuns = Jp532*q/JphSQ;
        VocMax300 = VocSQ300 + kb*300/q*np.log(Jp532*q/JphSQ)
        VocMax350 = VocSQ350 + kb*350/q*np.log(Jp532*q/JphSQ)
        TotalPL = np.mean(-E[1:-1]+E[0:-2])/2*(Ipl[0]+Ipl[-1]+2*np.sum(Ipl[1:-2]))
        TotalPL_Eg = np.mean(-E[1:maxI_idx]+E[0:maxI_idx-1])/2*(Ipl[0]+Ipl[maxI_idx]+2*np.sum(Ipl[1:maxI_idx-1]))
        PLQY = TotalPL/Jp532
        dmu_PLQY = VocMax350-kbeV*350*np.log(1/PLQY)
        chi_PLQY = dmu_PLQY/VocMax300 
        chi_PLQY_Eg = (VocMax350-kbeV*350*np.log(1/(TotalPL_Eg/Jp532)))/VocMax300
        PLQY_Eg = TotalPL_Eg/Jp532
        dmu_PLQY_Eg = VocMax350-kbeV*350*np.log(1/(TotalPL_Eg/Jp532))
        mean_Ipl = np.sum(Ipl_raw2*E)/np.sum(Ipl_raw2)
        """
    else: #maps
        k = 0
        while np.isnan(aipl_data[0,k]):
            k = k + 1
        lam = aipl_data[0,k:]
        E = heV*c/(lam*1e-9)
        mean_Ipl = np.zeros(aipl_data.shape[0])
        peak_pos = np.zeros(aipl_data.shape[0])
        FWHM = np.zeros(aipl_data.shape[0])
        PLQY = np.zeros(aipl_data.shape[0])
        dmu_PLQY = np.zeros(aipl_data.shape[0])
        chi_PLQY = np.zeros(aipl_data.shape[0])
        dmu_PLQY_Eg = np.zeros(aipl_data.shape[0])
        chi_PLQY_Eg = np.zeros(aipl_data.shape[0])
        for ii in range(1,aipl_data.shape[0]):
            Ipl = aipl_data[ii,k:]*(10**ND_filter)
            maxI = np.max(Ipl)
            maxI_idx = np.argmax(Ipl)
            #print(raw_data)
            if (raw_data[ii,k+maxI_idx] < 50) or (raw_data[ii,k+maxI_idx] > 59000):
                mean_Ipl[ii-1] = np.nan
                PLQY[ii-1] = np.nan
                dmu_PLQY[ii-1] = np.nan
                chi_PLQY[ii-1] = np.nan
                chi_PLQY_Eg[ii-1] = np.nan
                dmu_PLQY_Eg[ii-1] = np.nan
            else:
                peak_pos[ii-1] = E[maxI_idx]
                try:
                    HHMax_idx = np.argmin(np.absolute(maxI/2-Ipl[:maxI_idx]))
                    LHMax_idx = np.argmin(np.absolute(maxI/2-Ipl[maxI_idx:]))
                    LHMax_idx = LHMax_idx+maxI_idx-1
                    FWHM[ii-1] = E[HHMax_idx]-E[LHMax_idx]
                except ValueError:
                    FWHM[ii-1] = 0
                try:
                    VocSQ300 = VocSQs300_fn(E[maxI_idx])
                    VocSQ350 = VocSQs350_fn(E[maxI_idx])
                    JphSQ = Jphs_fn(E[maxI_idx])
                    NSuns = Jp532*q/JphSQ;
                    VocMax300 = VocSQ300 + kb*300/q*np.log(Jp532*q/JphSQ)
                    VocMax350 = VocSQ350 + kb*350/q*np.log(Jp532*q/JphSQ)
                    TotalPL = np.mean(-E[1:-1]+E[0:-2])/2*(Ipl[0]+Ipl[-1]+2*np.sum(Ipl[1:-2]))
                    TotalPL_Eg = np.mean(-E[1:maxI_idx]+E[0:maxI_idx-1])/2*(Ipl[0]+Ipl[maxI_idx]+2*np.sum(Ipl[1:maxI_idx-1]))
                    PLQY[ii-1] = TotalPL/Jp532
                    dmu_PLQY[ii-1] = VocMax350-kbeV*350*np.log(1/PLQY[ii-1])
                    chi_PLQY[ii-1] = dmu_PLQY[ii-1]/VocMax300 
                    chi_PLQY_Eg[ii-1] = (VocMax350-kbeV*350*np.log(1/(TotalPL_Eg/Jp532)))/VocMax300
                    PLQY_Eg = TotalPL_Eg/Jp532
                    dmu_PLQY_Eg[ii-1] = VocMax350-kbeV*350*np.log(1/(TotalPL_Eg/Jp532))
                    mean_Ipl[ii-1] = np.sum(Ipl*E)/np.sum(Ipl)
                except ValueError:
                    VocSQ300 = 0
                    VocSQ350 = 0
                    JphSQ = 0
                    NSuns = 1        

    
    
    
    return (E,Ipl)




In [40]:
if not os.path.exists(directory + '/out'):
    os.makedirs(directory + '/out')
        
for name1 in names_InGaAs:
    lname = directory + '/' + name1
    dname = []
    grating = '1200nm-InGaAs'
    elpl = 1 #0=PL, 1=EL
    ND_filter = 0
    laser_power = 40e-9
    (E,Ipl) = AIPL_maps(lname,dname,grating,laser_power,ND_filter,elpl)
    data_out = np.array([E,Ipl])
    np.savetxt(directory + '/out/' + lname[12:-4] + '_absoulte_units.txt',data_out)

for name2 in names_Si:
    lname = directory + '/' + name2
    dname = []
    grating = '1200nm'
    elpl = 1 #0=PL, 1=EL
    ND_filter = 0
    laser_power = 40e-9
    (E,Ipl) = AIPL_maps(lname,dname,grating,laser_power,ND_filter,elpl)
    data_out = np.array([E,Ipl])
    np.savetxt(directory + '/out/' + lname[12:-4] + '_absoulte_units.txt',data_out)

In [33]:
name1

'2018_7-24_Sample1_MW_1p5V_absoulte_units.txt'

In [19]:
data_out.shape

(2, 730)

In [ ]:
BB_raw_photon_data.shape

In [ ]:
BB850 = np.loadtxt('../../data/PLdata/grating_calibration_files/BB 850C 10 um hole D0 10x 150 grating CCD 532 nm NoDS.txt')
BB950 = np.loadtxt('../../data/PLdata/grating_calibration_files/BB 950C 10 um hole D0 10x 150 grating CCD 532 nm NoDS.txt')
BB1050 = np.loadtxt('../../data/PLdata/grating_calibration_files/BB 1050C 10 um hole D0 10x 150 grating CCD 532 nm NoDS.txt')

BB_raw_photon_data_1 = BB850[:,1]/np.insert(BB1050[1:,0]-BB1050[:-1,0], 
                                   0,BB1050[1,0]-BB1050[0,0])
BB_raw_photon_data_2 = BB950[:,1]/np.insert(BB1050[1:,0]-BB1050[:-1,0], 
                                   0,BB1050[1,0]-BB1050[0,0])
BB_raw_photon_data_3 = BB1050[:,1]/np.insert(BB1050[1:,0]-BB1050[:-1,0], 
                                   0,BB1050[1,0]-BB1050[0,0])

In [39]:
name1 = '2018_7-24_Sample10_MW_1p1V_Sidetector.txt'
name2 = directory + '/' + name1
name2[12:-4]

'2018_7-24_Sample10_MW_1p1V_Sidetector'

In [ ]:
aa = np.array([BB_raw_photon_data_1,BB_raw_photon_data_2,BB_raw_photon_data_3])

In [ ]:
aa.shape

In [ ]:
bb = np.mean(aa,0)

In [ ]:
bb.shape

In [ ]:
AbsFluxesPerNM = np.zeros(aa.shape)

AbsFluxesPerNM.shape

In [ ]:
AbsFluxesPerNM

In [ ]:
dark = 1
dark != []

In [41]:
data_out.shape

(2, 933)